In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

data = pd.read_csv("output.csv")
train_dates = pd.to_datetime(data["Date"])
train_dates = train_dates[7:]
train_dates


7      1988-08-10
8      1988-08-11
9      1988-08-12
10     1988-08-15
11     1988-08-16
          ...    
9009   2023-08-23
9010   2023-08-24
9011   2023-08-25
9012   2023-08-27
9013   2023-08-28
Name: Date, Length: 9007, dtype: datetime64[ns]

In [81]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [132]:
from copy import deepcopy as dc

def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    df.set_index('Date', inplace=True)

    for i in range(1, n_steps+1):
        df[f'Price(t-{i})'] = df['Price'].shift(i)

    df.dropna(inplace=True)

    return df

lookback = 7
shifted_df = prepare_dataframe_for_lstm(data, lookback)
shifted_df

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7)
Date,,,,,,,,
1988-08-10,0.8875,0.8760,0.8710,0.8870,0.8780,0.8680,0.8600,0.8655
1988-08-11,0.9010,0.8875,0.8760,0.8710,0.8870,0.8780,0.8680,0.8600
1988-08-12,0.8835,0.9010,0.8875,0.8760,0.8710,0.8870,0.8780,0.8680
1988-08-15,0.8880,0.8835,0.9010,0.8875,0.8760,0.8710,0.8870,0.8780
1988-08-16,0.8880,0.8880,0.8835,0.9010,0.8875,0.8760,0.8710,0.8870
...,...,...,...,...,...,...,...,...
2023-08-23,3.8375,3.7885,3.7185,3.7060,3.6915,3.6575,3.6675,3.7255
2023-08-24,3.7935,3.8375,3.7885,3.7185,3.7060,3.6915,3.6575,3.6675
2023-08-25,3.7840,3.7935,3.8375,3.7885,3.7185,3.7060,3.6915,3.6575


In [83]:
shifted_df_as_np_before_scaling = shifted_df.to_numpy()
shifted_df_as_np_before_scaling

array([[0.8875, 0.876 , 0.871 , ..., 0.868 , 0.86  , 0.8655],
       [0.901 , 0.8875, 0.876 , ..., 0.878 , 0.868 , 0.86  ],
       [0.8835, 0.901 , 0.8875, ..., 0.887 , 0.878 , 0.868 ],
       ...,
       [3.784 , 3.7935, 3.8375, ..., 3.706 , 3.6915, 3.6575],
       [3.8027, 3.784 , 3.7935, ..., 3.7185, 3.706 , 3.6915],
       [3.788 , 3.8027, 3.784 , ..., 3.7885, 3.7185, 3.706 ]])

In [84]:
shifted_df_as_np_before_scaling.shape

(9007, 8)

In [85]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(shifted_df_as_np_before_scaling)
shifted_df_as_np = scaler.transform(shifted_df_as_np_before_scaling)
shifted_df_as_np
#shifted_df_as_np

array([[-0.64423604, -0.64890486, -0.65093478, ..., -0.65215273,
        -0.6554006 , -0.65316769],
       [-0.63875525, -0.64423604, -0.64890486, ..., -0.64809289,
        -0.65215273, -0.6554006 ],
       [-0.64585998, -0.63875525, -0.64423604, ..., -0.64443903,
        -0.64809289, -0.65215273],
       ...,
       [ 0.53169722,  0.53555407,  0.55341737, ...,  0.50003045,
         0.49414368,  0.48034021],
       [ 0.53928912,  0.53169722,  0.53555407, ...,  0.50510525,
         0.50003045,  0.49414368],
       [ 0.53332115,  0.53928912,  0.53169722, ...,  0.53352415,
         0.50510525,  0.50003045]])

In [86]:
X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

X.shape, y.shape

((9007, 7), (9007,))

In [87]:
X = dc(np.flip(X, axis=1))
X

array([[-0.65316769, -0.6554006 , -0.65215273, ..., -0.64443903,
        -0.65093478, -0.64890486],
       [-0.6554006 , -0.65215273, -0.64809289, ..., -0.65093478,
        -0.64890486, -0.64423604],
       [-0.65215273, -0.64809289, -0.64443903, ..., -0.64890486,
        -0.64423604, -0.63875525],
       ...,
       [ 0.48034021,  0.49414368,  0.50003045, ...,  0.53352415,
         0.55341737,  0.53555407],
       [ 0.49414368,  0.50003045,  0.50510525, ...,  0.55341737,
         0.53555407,  0.53169722],
       [ 0.50003045,  0.50510525,  0.53352415, ...,  0.53555407,
         0.53169722,  0.53928912]])

In [88]:
X = X.reshape((-1, lookback, 1))
y = y.reshape((-1, 1))
X.shape,y.shape

((9007, 7, 1), (9007, 1))

In [89]:
X = torch.tensor(X).float()
y = torch.tensor(y).float()

In [90]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

dataset = TimeSeriesDataset(X, y)

In [91]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [92]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 7, 1]) torch.Size([16, 1])


In [93]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

LSTM(
  (lstm): LSTM(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [94]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [95]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [96]:
learning_rate = 0.001
num_epochs = 20
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
"""
Adam does the gradient descent for me
"""
for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1
Batch 100, Loss: 0.578
Batch 200, Loss: 0.348
Batch 300, Loss: 0.089
Batch 400, Loss: 0.011
Batch 500, Loss: 0.007
Val Loss: 0.005
***************************************************

Epoch: 2
Batch 100, Loss: 0.004
Batch 200, Loss: 0.002
Batch 300, Loss: 0.002
Batch 400, Loss: 0.002
Batch 500, Loss: 0.001
Val Loss: 0.001
***************************************************

Epoch: 3
Batch 100, Loss: 0.001
Batch 200, Loss: 0.001
Batch 300, Loss: 0.001
Batch 400, Loss: 0.001
Batch 500, Loss: 0.001
Val Loss: 0.001
***************************************************

Epoch: 4
Batch 100, Loss: 0.001
Batch 200, Loss: 0.001
Batch 300, Loss: 0.001
Batch 400, Loss: 0.001
Batch 500, Loss: 0.001
Val Loss: 0.001
***************************************************

Epoch: 5
Batch 100, Loss: 0.001
Batch 200, Loss: 0.001
Batch 300, Loss: 0.001
Batch 400, Loss: 0.001
Batch 500, Loss: 0.001
Val Loss: 0.001
***************************************************

Epoch: 6
Batch 100, Loss: 0.001
Bat

In [97]:
from datetime import date, timedelta
start_date = (train_dates.iloc[-1].date()+ timedelta(days=1)).strftime("%Y-%m-%d")
end_date = '2023-12-31'
test_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with the date column
testset = pd.DataFrame({'Date': test_dates})
testset['Date'] = pd.to_datetime(testset['Date'])
testset.to_numpy()
testset


,Date
0,2023-08-29
1,2023-08-30
2,2023-08-31
3,2023-09-01
4,2023-09-02
...,...
120,2023-12-27
121,2023-12-28
122,2023-12-29
123,2023-12-30


In [98]:
min_data = np.min(shifted_df_as_np_before_scaling)
max_data = np.max(shifted_df_as_np_before_scaling)
scaled_min = np.min(shifted_df_as_np)
scaled_max = np.max(shifted_df_as_np)

In [99]:


def custom_scaling(data, min_val, max_val):
    global min_data, max_data
    if np.min(data) < min_data:
        min_data = np.min(data)
    else:
        min_data = np.min(shifted_df_as_np_before_scaling)
    if np.max(data) > max_data:
        max_data = np.max(data)
    else:
        max_data = np.max(shifted_df_as_np_before_scaling)
    scaled_data = min_val + ((data - min_data) / (max_data - min_data)) * (max_val - min_val)
    return scaled_data

scaled_min = np.min(shifted_df_as_np)
scaled_max = np.max(shifted_df_as_np)
print(scaled_max)
def reverse_custom_scaling(scaled_data, min_val, max_val):
    global scaled_min, scaled_max
    if np.min(scaled_data) < scaled_min:
        scaled_min = np.min(scaled_data)
    if np.max(scaled_data) > scaled_max:
        scaled_max = np.max(scaled_data)
    original_data = min_data + (((scaled_data - min_val)/(max_val - min_val))*(max_data-min_data))
    return original_data

0.9999999999999998


array([[3.83351183, 3.788     , 3.8027    , 3.784     , 3.7935    ,
        3.8375    , 3.7885    ]])

In [141]:
for _ in range(test_dates.shape[0]):    
    shifted_df_as_np_before_scaling = shifted_df.to_numpy()
    sample2 = shifted_df_as_np_before_scaling[-1,1:]
    sample2 = sample2.reshape(1,-1)
    sample2_np = np.flip(sample2,axis=1)
    sample2_np = custom_scaling(sample2_np,-1,1)
    sample2_np = sample2_np.reshape((-1, lookback, 1))
    sample2_tensor = torch.tensor(sample2_np).float()
    with torch.no_grad():
        predicted = model(sample2_tensor.to(device)).to('cpu').numpy()
    #predicted
    res = reverse_custom_scaling(predicted,-1,1)
    sample2[0][-1]
    new_row = {'Price':res[0][0], 
               'Price(t-1)':shifted_df_as_np_before_scaling[-1][0],
               'Price(t-2)':shifted_df_as_np_before_scaling[-1][1],
               'Price(t-3)':shifted_df_as_np_before_scaling[-1][2],
               'Price(t-4)':shifted_df_as_np_before_scaling[-1][3],
               'Price(t-5)':shifted_df_as_np_before_scaling[-1][4],
               'Price(t-6)':shifted_df_as_np_before_scaling[-1][5],
               'Price(t-7)':shifted_df_as_np_before_scaling[-1][6]
               }
    new_row 
    shifted_df = shifted_df._append(new_row,ignore_index=True)
shifted_df 

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7)
0,0.887500,0.876000,0.871000,0.887000,0.878000,0.868000,0.860000,0.865500
1,0.901000,0.887500,0.876000,0.871000,0.887000,0.878000,0.868000,0.860000
2,0.883500,0.901000,0.887500,0.876000,0.871000,0.887000,0.878000,0.868000
3,0.888000,0.883500,0.901000,0.887500,0.876000,0.871000,0.887000,0.878000
4,0.888000,0.888000,0.883500,0.901000,0.887500,0.876000,0.871000,0.887000
...,...,...,...,...,...,...,...,...
9130,4.883018,4.882437,4.881827,4.881187,4.880517,4.879812,4.879074,4.878299
9131,4.883573,4.883018,4.882437,4.881827,4.881187,4.880517,4.879812,4.879074
9132,4.884100,4.883573,4.883018,4.882437,4.881827,4.881187,4.880517,4.879812
9133,4.884603,4.884100,4.883573,4.883018,4.882437,4.881827,4.881187,4.880517


In [150]:
plt.plot(testset,shifted_df.tail(125)['Price'].to_numpy(), label='Predicted Price')
#plt.plot(train_dates,predicted, label='Predicted Price')

plt.xlabel('Dates')
plt.ylabel('Price')
plt.legend()
plt.show()


In [151]:
testset

,Date
0,2023-08-29
1,2023-08-30
2,2023-08-31
3,2023-09-01
4,2023-09-02
...,...
120,2023-12-27
121,2023-12-28
122,2023-12-29
123,2023-12-30


array([3.86124492, 3.88101578, 3.89498162, 3.91245747, 3.92832112,
       3.94502401, 3.96163893, 3.97821832, 3.99499583, 4.01167774,
       4.02849865, 4.04526138, 4.06209278, 4.07889271, 4.09571028,
       4.11250257, 4.12927771, 4.14601851, 4.16271734, 4.17936516,
       4.19595003, 4.2124629 , 4.2288928 , 4.2452302 , 4.26146317,
       4.27758265, 4.29357672, 4.3094368 , 4.32515144, 4.34071016,
       4.35610437, 4.37132311, 4.38635778, 4.40119839, 4.41583586,
       4.43026304, 4.44446993, 4.45845079, 4.47219706, 4.48570251,
       4.49896049, 4.51196575, 4.52471256, 4.53719664, 4.54941416,
       4.56135988, 4.57303381, 4.58443022, 4.59555006, 4.60639   ,
       4.61695004, 4.62723017, 4.6372304 , 4.64695168, 4.65639496,
       4.66556168, 4.67445517, 4.68307686, 4.69143009, 4.69951868,
       4.70734501, 4.71491432, 4.72222996, 4.72929716, 4.73611927,
       4.74270344, 4.74905205, 4.75517321, 4.76106977, 4.76674891,
       4.77221537, 4.7774744 , 4.78253317, 4.78739548, 4.79206